<a href="https://colab.research.google.com/github/tvs-dk/data-analysis/blob/main/linear_int_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This block of code can be used to make an example file. Can be ignored if you already have a datafile.

In [ ]:
import random

# Generate random time and data
time = [i for i in range(0, 601, 10)]
data = [random.randint(0, 100) for i in range(len(time))]

# Write to file
with open('example_input.csv', 'w') as f:
    for t, d in zip(time, data):
        f.write(f'{t},{d}\n')


Use the block of code below to upload a file

In [1]:
from google.colab import files
uploaded = files.upload()


Saving 0_mM_NaCl.csv to 0_mM_NaCl.csv
Saving 10_mM_NaCl.csv to 10_mM_NaCl.csv
Saving 25_mM_NaCl.csv to 25_mM_NaCl.csv
Saving 50_mM_NaCl.csv to 50_mM_NaCl.csv


The data file needs to be in csv format without headers.
The block of code below performs the linear interpolation.


In [ ]:
import pandas as pd

# Read in data file
data = pd.read_csv('input_file.csv', header=None, names=['time', 'data'], dtype={'time': float})

# Convert time to minutes
data['time'] /= 60

# Perform linear interpolation at 2-minute intervals
new_time = pd.Series(range(0, int(data['time'].max()) + 2, 2))
new_data = pd.Series(pd.Series(new_time).interpolate(method='linear', 
                                                     limit_direction='both'))

# Combine new time and data into a new DataFrame
new_data = pd.DataFrame({'time': new_time, 'data': pd.Series(new_time).map(data.set_index('time')['data']).interpolate(method='linear', limit_direction='both')})

# Output new data file
new_data.to_csv('output_file_new_correct.csv', header=None, index=False)


Below is an updated version that runs on all csv files in the current directory

In [9]:
import pandas as pd
import os

# Create output directory if it doesn't exist
if not os.path.exists('output'):
    os.makedirs('output')

# Iterate over all CSV files in current directory
for filename in os.listdir('.'):
    if filename.endswith('.csv'):
        # Read in data file
        data = pd.read_csv(filename, header=0, names=['time', 'data'], dtype={'time': float})
        #data = pd.read_csv(filename, header=0, dtype={'time': float})
        # Convert time to minutes
        data['time'] /= 60

        # Perform linear interpolation at 2-minute intervals
        new_time = pd.Series(range(0, int(data['time'].max()) + 2, 2))
        new_data = pd.Series(new_time).map(data.set_index('time')['data']).interpolate(method='linear', limit_direction='both')

        # Combine new time and data into a new DataFrame
        new_data = pd.DataFrame({'time': new_time, 'data': new_data})

        # Output new data file
        output_filename = os.path.join('output', filename[:-4] + '-output.csv')
        new_data.to_csv(output_filename, header=None, index=False)
